## xiier虚拟人后端接口, 使用vicuna-13b作为大模型支持openai的api, 添加中文asr接口, 中文TTS接口, 中文sentiment分析接口, 方便单机部署虚拟人项目

In [ ]:
#@title 播放一个音频,让colab长时间运行不自动断开连接 { display-mode: "form" }

#@markdown 运行后让音频一直播放:
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

#### 检查设备环境, 并且下载接口代码

In [ ]:
!/opt/bin/nvidia-smi

#### 下载项目代码安装全部环境依赖

In [ ]:
!git clone https://github.com/xhb/xiier-fastchat.git
%cd /content/xiier-fastchat
!pip install -e .

In [ ]:
%cd /content/xiier-fastchat/fastchat/vits_tts_zh/
!pip install -r ./vits/requirements.txt


In [ ]:
%cd /content/xiier-fastchat/fastchat/vits_tts_zh/vits/monotonic_align
!mkdir monotonic_align
!python setup.py build_ext --inplace
%cd /content/xiier-fastchat

#### 安装稳定版本的llama Gptq模型支持库

In [ ]:
!mkdir repositories
%cd repositories
!git clone https://github.com/oobabooga/GPTQ-for-LLaMa.git -b cuda
%cd GPTQ-for-LLaMa
!python setup_cuda.py install
%cd ../..

#### 下载 vicuna-13GPTQ模型, 中文语音识别 asr_zh 模型, 中文语音生成 vits_zh 模型, 中文情感色彩分析模型

In [ ]:
!python download-model.py anon8231489123/vicuna-13b-GPTQ-4bit-128g

In [ ]:
!python download-model.py xiier/asr_zh

In [ ]:
!python download-model.py xiier/vits_zh

In [ ]:
!python download-model.py xiier/sentiment_cls

In [ ]:
!sudo apt install ffmpeg --fix-missing && ffmpeg -version

#### 构建openai官方接口, 中文语音识别接口, 中文文字朗读TTS接口, 中文情感色彩分析接口

In [ ]:
!sudo apt-get install screen

In [ ]:
!screen -S controller -dm python -m fastchat.serve.controller --host "127.0.0.1" --port 21001

在这一行执行之后等待大概30s,等待gpu的显存加载模型完成, 再启动web服务

In [ ]:
!screen -S controller -dm python -m fastchat.serve.model_worker --model-path "anon8231489123/vicuna-13b-GPTQ-4bit-128g" --model-name "text-embedding-ada-002" --gptq-wbits 4 --gptq-groupsize 128 --host "127.0.0.1" --port 21002 --worker-address "http://127.0.0.1:21002" --controller-address "http://127.0.0.1:21001"

#### 使用 ngrok 把本地的http服务代理到公共url

In [ ]:
!pip install pyngrok

In [ ]:
from pyngrok import ngrok
port = 5555
ngrok.set_auth_token("2QRC3hph4qKZrV5aSXPHylX5hzF_4iinXrMr6VM4tJir9iGEK")
public_url = ngrok.connect(port).public_url

print("baseUrl: \"{}\"".format(public_url))


In [ ]:
!python -m fastchat.serve.openai_api_server --controller-address "http://127.0.0.1:21001" --host "0.0.0.0" --port 5555

#### debug: ***启动调试终端开始调试 (调试可用)***

In [ ]:
!pip install colab-xterm
%load_ext colabxterm

In [ ]:
%xterm